## Download outlines for the barrier islands located in NC, US

using data from: Mulhern, Julia S., Johnson, Cari A., and John M. Martin 2021 Dataset for: Is barrier island morphology a function of tidal and wave regime? The Hive: University of Utah Research Data Repository https://hive.utah.edu/concern/datasets/cf95jb516?locale=en.¶

In [1]:
### Imports

import os
import urllib
import requests
import zipfile
import geopandas as gpd
import shutil

In [2]:
### Set working directory

path='..' # introduce path to your working directory
os.chdir(path)

### Dowload barrier island outlines from Mulhern et al., 2021

# Create folder if it does no exist
outdir= './data/Barriers'
if not os.path.exists(outdir):
    os.makedirs(outdir)

# Retrive zip file from repository and save it
url = 'https://hive.utah.edu/downloads/7d278t05z'
r = requests.get(url, allow_redirects=True)
open('./data/Barriers/Barriers.zip', 'wb').write(r.content)

# Extract files
with zipfile.ZipFile('./data/Barriers/Barriers.zip', 'r') as zip_ref:
    zip_ref.extractall('./data/Barriers/')

# Barrier outlines are duplicated, remove duplicated files
my_dir = './data/Barriers/Coastal_Morph_Shapefiles_Exported_2020_06_01' 
for fname in os.listdir(my_dir):
    if fname.endswith("xy_.dbf") | fname.endswith("xy_.shp") | fname.endswith("xy_.shx"):
        os.remove(os.path.join(my_dir, fname))

In [3]:
### Identify which barrier islands from Mulhern et al. are located in NC, USA and move them to a new folder

rootdir = './data/Barriers/Coastal_Morph_Shapefiles_Exported_2020_06_01'

# Create folder if it does no exist
outdir= './data/Barriers/NC_Barriers'
if not os.path.exists(outdir):
    os.makedirs(outdir)

AG_states=['NC'] # code for NC

# Iterate through folder files and check if they start with the AG codes. If True, move file to a different folder
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        file_path = os.path.join(subdir, file)
        if file.startswith(tuple(AG_states)):  
            shutil.move(file_path, os.path.join(outdir, os.path.basename(file_path))) 

In [4]:
### Downloaded polygons do not include a prj file, so crs (WGS84) needs to be defined

rootdir = './data/Barriers/NC_Barriers'
extensions = ('.shp')

# Iterate through folder containing the shapefiles, set crs and save shp so they have a prj file. 
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            name= file.replace('.shp','')
            data= gpd.read_file(file_path)
            data= data.set_crs(epsg=4326)
            data.to_file(file_path)

In [5]:
### Create 200 m buffer around barrier outlines to clip CUDEM tiles and make sure that all network nodes have elevation data

# Loop within polygons folder and create a 200m buffer to increase the size of each barrier.  
rootdir = './data/Barriers/NC_Barriers'
extensions = ('.shp')
outdir = './data/Barriers/Buffers_200m'

# Create folder if it does no exist
if not os.path.exists(outdir):
    os.makedirs(outdir)

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        ext = os.path.splitext(file)[-1].lower()
        if ext in extensions:
            file_path = os.path.join(subdir, file)
            name= file.replace('.shp','')
            # read polygons
            barrier = gpd.read_file(file_path)
            # change crs to projected
            barrier = barrier.to_crs('esri:102003')
            # buffer 200m
            barrier2 = barrier['geometry'] = barrier.geometry.buffer(200)
            # reproject to original crs
            barrier2= barrier2.to_crs(epsg=4326)
            # vave file
            barrier2.to_file('./data/Barriers/Buffers_200m/{0}.shp'.format(name))